In [1]:
import re
import numpy as np
from itertools import groupby
from functools import reduce
from operator import mul

In [2]:
with open('day20_input.txt', 'r') as f:
    data = [line.rstrip('\n') for line in f]
data = [list(group) for k, group in groupby(data, lambda x: x == "") if not k]
data = [(int(re.findall(r'\d+', image[0])[0]), 
         np.array([[char for char in line] for line in image[1:]])) for image in data]

In [3]:
def get_edges(arr):
    edges = {''.join(arr[0]), ''.join(arr[0][::-1]),
             ''.join(arr[-1]), ''.join(arr[-1])[::-1],
             ''.join(arr[:, 0]), ''.join(arr[:, 0])[::-1],
             ''.join(arr[:, -1]), ''.join(arr[:, -1])[::-1]}
    return edges

In [4]:
adj = dict(zip([i[0] for i in data], [0] * len(data)))
for i in range(len(data)):
    id_i, a_i = data[i]
    i_edges = get_edges(a_i)
    for j in range(i + 1, len(data)):
        id_j, a_j = data[j]
        j_edges = get_edges(a_j)
        if len(i_edges.intersection(j_edges)) > 0:
            adj[id_i] += 1
            adj[id_j] += 1

In [5]:
corners = []
for id_, count in adj.items():
    if count < 3:
        corners.append(id_)
        print(id_, count)

3467 2
3169 2
1019 2
1249 2


In [6]:
reduce(mul, corners)

13983397496713

In [7]:
def find_fitting(border, border_orient, arr):
    for i in range(4):
        arr = np.rot90(arr)
        for j in range(2):
            arr = np.flip(arr, axis=0)
            if (arr[border_orient] == border).all():
                return arr, i + 1, j + 1
    return None, None, None

In [8]:
# id 1019 found as top-left through trial and error
top_left_idx = [item[0] for item in data].index(1019)

In [9]:
data.insert(0, data.pop(top_left_idx))
images = [[data.pop(0)[1]]]

In [10]:
#First row
for i in range(12):
    img = images[0][i]
    for j in range(len(data)):
        _, aj = data[j]
        arr, n_rot, n_flips = find_fitting(img[:, -1], (slice(None), 0), aj)
        if arr is not None:
            images[0].append(arr)
            data.pop(j)
            break

In [11]:
#Rest of rows
for n in range(11):
    images.append([])
    for i in range(12):
        img = images[n][i]
        for j in range(len(data)):
            _, aj = data[j]
            arr, n_rot, n_flips = find_fitting(img[-1], (0, slice(None)), aj)
            if arr is not None:
                images[n + 1].append(arr)
                data.pop(j)
                break

In [12]:
true_image = []
for row in images:
    true_row = []
    for img in row:
        true_row.append(img[1:-1, 1:-1])
    true_row = np.concatenate(true_row, axis=1)
    true_image.append(true_row)
true_image = np.concatenate(true_image, axis=0)

In [13]:
monster = ['                  # ', '#    ##    ##    ###', ' #  #  #  #  #  #   ']
monster = np.array([[char for char in line] for line in monster])

In [14]:
monster

array([[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', '#', ' '],
       ['#', ' ', ' ', ' ', ' ', '#', '#', ' ', ' ', ' ', ' ', '#', '#',
        ' ', ' ', ' ', ' ', '#', '#', '#'],
       [' ', '#', ' ', ' ', '#', ' ', ' ', '#', ' ', ' ', '#', ' ', ' ',
        '#', ' ', ' ', '#', ' ', ' ', ' ']], dtype='<U1')

In [15]:
mask = monster == '#'
vals = np.array(['#'] * mask.sum())

In [17]:
import skimage

In [24]:
arr = true_image
n_monsters = 0
for i in range(4):
    arr = np.rot90(arr)
    for j in range(2):
        arr = np.flip(arr, axis=0)
        windows = skimage.util.view_as_windows(arr, mask.shape)
        for n, col in enumerate(windows):
            for k, row in enumerate(col):
                if np.all(row[mask] == vals, axis = -1):
                    n_monsters += 1

In [25]:
(true_image == '#').sum() - n_monsters * 15

2424